<a href="https://colab.research.google.com/github/MERTZGZ/Vehicle_detection/blob/main/Vehicle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/custom_object_detection/yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5


In [ ]:
%cd yolov5

In [ ]:
%pip install -qr requirements.txt  # install

In [ ]:
import torch

In [ ]:
import utils

In [ ]:
display = utils.notebook_init()  # checks

In [ ]:
!python train.py --img 128 --batch 128 --epochs 500 --data custom_data.yaml --weights last.pt  --patience 0

In [ ]:
# Test dosyasindaki görselleri bu kod parçasından direk detect ettiginizde otamatik olarak results dosyasını günceller.

import os
import time
import torch
import pandas as pd


# Test klasörünün yolunu verin
test_folder_path = '/content/drive/MyDrive/custom_object_detection/test'

# Ölçmek istediğiniz modelin yolunu verin
model_path = '/content/drive/MyDrive/custom_object_detection/yolov5/yolov5/best.pt'

# Sonuçların kaydedileceği txt dosyasının yolunu ve adını verin
results_file = '/content/drive/MyDrive/custom_object_detection/results.txt'

# Modeli yükleyin
model = torch.hub.load('/content/drive/MyDrive/custom_object_detection/yolov5/yolov5','custom', path=model_path, force_reload=True, source='local')

model.conf = 0.50 # yeterli zaman olmadigi icin model cok fazla egitilememistir o yuzden results txt de test jpg leri uzerinde tespit oldugu gosterilsin diye esik degeri dusuk tutlmustur!!!.
model.iou = 0.50  # yeterli zaman olmadigi icin model cok fazla egitilememistir o yuzden results txt de test jpg leri uzerinde tespit oldugu gosterilsin diye esik degeri dusuk tutlmustur!!!.

# Test klasöründeki resimlerin listesini oluşturun
test_images = os.listdir(test_folder_path)

# Dosya üzerine yazmak için açın
f = open(results_file, 'w')

for image_name in test_images:
    # Resmin tam yolunu oluşturun
    image_path = os.path.join(test_folder_path, image_name)

    # Zamanı ölçün
    start_time = time.time()

    # Resmi yükleyin
    img = image_path

    # İnceleme yapın
    results = model(img)

    # Sonuçları pandas DataFrame'ine çevirin
    df = results.pandas().xyxy[0]


    # Zamanı ölçün
    end_time = time.time()
    elapsed_time = (end_time - start_time) * 1000

    # Sonuçları dosyaya yazın
    formatted_time = round(elapsed_time, 2) #Ondalıklı bir sayının virgülden sonra iki basamağını almak 

    #Bu kod ile, veri çerçevesini oluştururken `pd.options.display.max_colwidth` özelliğini değiştirin ve txt dosyasına yazdırırken veri çerçevesinin sütunlarının genişliğini değiştirin. Bu sayede, `name` sütunu bir alt satırda yer almayacak ve txt dosyasında düzgün bir şekilde görüntülenecektir.
    pd.options.display.max_colwidth = 100
    #Bu kod, name sütununun her bir değerinin uzunluğunu 100 karakterden azaltacaktır. Bu sayede, txt dosyasında görüntülenen name sütunu daha düzgün görünecektir. Not: Bu önerilen çözüm, name sütununun değerlerinin 100 karakterden az olduğu durumlarda da çalışacaktır. Eğer name sütununun değerleri 100 karakterden azsa, bu kodun etkisi olmayacaktır.
    df['name'] = df['name'].str[:100]

    f.write(image_name + ": " + str(formatted_time) + "ms"+ " Hızında tespit edilmiştir." + "\n" + "\n")  

    f.write(df.to_string() + "\n" + "\n")


# Dosyayı kapatın
f.close()
